In [ ]:
from astropy.nddata import CCDData, overlap_slices
from ccdproc import ImageFileCollection

In [ ]:
import arya
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("../../imaging")
from convenience_functions import show_image, show_image_residual

In [ ]:
sys.path.append("..")
from phot_utils import swap_byteorder

In [ ]:
from pathlib import Path

In [ ]:
from astropy.io import fits

In [ ]:
import astropy
from astropy.table import Table

In [ ]:
objname = "yasone2"
filt = "i"

In [ ]:
# cat = Table.read(f"../{objname}/stacked_{filt}/detection.cat", hdu=2)

In [ ]:
plt.close()

In [ ]:
img.wcs.has_distortion

In [ ]:
frames_orig = []
frames_resamp = []
frames_updated = []
frames_uncorr = []
basename_corrected = "scamped"

for i in range(1, 16):
    img = CCDData.read(f"../{objname}/img_{filt}_{i:02d}/flat_fielded-astrom.fits")
    frames_orig.append(img)

    img0 = CCDData.read(f"../{objname}/img_{filt}_{i:02d}/flat_fielded.fits")
    frames_uncorr.append(img0)

    img2 = CCDData.read(f"../{objname}/img_{filt}_{i:02d}/{basename_corrected}.fits", unit="adu")
    # head = fits.Header.fromtextfile(f"../{objname}/img_{filt}_{i:02d}/nobkg.head")
    # img2.wcs = astropy.wcs.WCS(head)
    print(img2.wcs.has_distortion)
    
    frames_updated.append(img2)

    if i == 1:
            img3 = CCDData.read(f"../{objname}/stacked_{filt}/{basename_corrected}.resamp.fits", unit="adu")
    else:
        img3 = CCDData.read(f"../{objname}/stacked_{filt}/{basename_corrected}_v{i}.resamp.fits", unit="adu")

    frames_resamp.append(img3)

    print("astrolign")
    print(img.wcs.to_header(relax=True).tostring(sep="\n"))
    print()
    
    print("scamp")
    print(img2.wcs.to_header(relax=True).tostring(sep="\n"))
    print()
    print("swarp")
    print(img3.wcs.to_header(relax=True).tostring(sep="\n"))
    print()
    print("-"*100)

In [ ]:
coadd = CCDData.read(f"../{objname}/stacked_{filt}/coadd.fits", unit="adu")

Failed matches
yasone 2 
/img_g_14
yasone2//img_g_12
processing yasone2//img_g_11 

In [ ]:
cat_gaia = Table.read(f"../{objname}/scamp_reference.fits")

In [ ]:
import astropy.units as u

In [ ]:
img.wcs.pixel_to_world(5, 5)

In [ ]:
# %matplotlib ipympl

In [ ]:
coords_gaia = astropy.coordinates.SkyCoord(cat_gaia["ra"], cat_gaia["dec"])

In [ ]:
# def show_image_sources(img, coords=coords,  **kwargs):
#     show_image(img, log=True, **kwargs)
#     x, y = img.wcs.world_to_pixel(coords)
#     plt.scatter(x, y, s=0.1, c="red")

In [ ]:
def plot_sources_in_wcs(img, coords, **kwargs):
    x, y = frames_updated[i].wcs.world_to_pixel(coords)
    plt.scatter(x, y, **kwargs)

In [ ]:
x, y = frames_orig[0].wcs.world_to_pixel(coords_gaia)

filt_gaia = (0 < x) & (x < 2048) & (0 < y) & (y < 2048)

In [ ]:
coords_good_gaia = coords_gaia[filt_gaia]

# Residuals

In [ ]:
def get_cat(i, objname=objname, filt=filt):
    return Table.read(f"../{objname}/img_{filt}_{i:02d}/detection.cat", hdu=2) 

In [ ]:
def get_img_julen(i, objname=objname, filt=filt):
    return CCDData.read(f"../{objname}/julen_stack_{filt}_{i:04d}/stacked.fits")

In [ ]:
def get_img_stack(objname=objname, filt=filt):
    # Table.read(f"../{objname}/stacked_{filt}/detection.cat", hdu=2), 
    return CCDData.read(f"../{objname}/stacked_{filt}/coadd.fits", unit="adu")

In [ ]:
sys.path.append("..")
from phot_utils import xmatch

In [ ]:
import numpy as np

In [ ]:
def plot_xmatch(cat, wcs, fig=None, axs=None):
    x, y = cat["XWIN_IMAGE"], cat["YWIN_IMAGE"]
    coords = wcs.pixel_to_world(x, y)
    if isinstance(coords, list):
        coords = astropy.coordinates.SkyCoord(coords[0] * u.deg, coords[1] * u.deg)
    idxs, dists, has_xmatch = xmatch(coords_good_gaia.ra, coords_good_gaia.dec, coords.ra, coords.dec, 5*u.arcsec)

    if fig is None:
        fig, axs=plt.subplots(1, 2, sharex=True, sharey=True, figsize=(5, 2.5))

    plt.sca(axs[0])
    plt.scatter(coords.ra, coords.dec, color="k", s=1)
    plt.scatter(coords_good_gaia.ra[has_xmatch], coords_good_gaia.dec[has_xmatch], marker="s")
    plt.scatter(coords_good_gaia.ra[~has_xmatch], coords_good_gaia.dec[~has_xmatch], marker="s", facecolor="none", edgecolor="red", s=3)

    plt.scatter(coords.ra[idxs[has_xmatch]], coords.dec[idxs[has_xmatch]], s=3)

    plt.sca(axs[1])
    x0 = coords.ra[idxs[has_xmatch]].value
    y0 = coords.dec[idxs[has_xmatch]].value
    x1 = coords_good_gaia.ra[has_xmatch].value
    y1 = coords_good_gaia.dec[has_xmatch].value
    
    # plt.quiver(x0, y0)#, (x1-x0)/u.degree, (y1-y0)/u.degree)
    p = plt.quiver(x0, y0, x1-x0, y1-y0, scale_units="xy", scale=1/5, units="xy")
    x_l, y_l = np.min(x0), np.max(y0) + 0.01
    plt.quiver(x_l, y_l, 5/3600, 0, scale_units="xy", scale=1/5, width=0.003)
    plt.text(x_l, y_l, "5\"")

In [ ]:
import sep

In [ ]:
def plot_xmatch_from_image(img):
    data = swap_byteorder(img.data)
    bkg = sep.Background(data, fw=6, fh=6, bw=128, bh=128)
    objects = sep.extract(data - bkg, 5, err=bkg.globalrms)
    cat = dict(XWIN_IMAGE=objects["x"], YWIN_IMAGE=objects["y"])
    plot_xmatch(cat, img.wcs)

In [ ]:
def plot_xmatch_both(idx):
    cat = get_cat(idx)
    wcs_orig = frames_orig[idx-1].wcs
    wcs_scamp = frames_updated[idx-1].wcs

    fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(5,5))

    p = plot_xmatch(cat, wcs_orig, fig=fig, axs=axs[0])

    plot_xmatch(cat, wcs_scamp, fig=fig, axs=axs[1])

In [ ]:
plot_xmatch_from_image(frames_uncorr[0])

In [ ]:

plot_xmatch_from_image(get_img_stack())

In [ ]:
img = get_img_julen(4)
plot_xmatch_from_image(img)

In [ ]:
for i in range(len(frames_updated)):
    plot_xmatch_from_image(frames_updated[i])
    plt.title(objname + " " + filt + " " + str(i+1))

In [ ]:
for i in range(len(frames_updated)):
    plot_xmatch_from_image(frames_orig[i])
    plt.title(objname + " " + filt + " " + str(i+1))

# Image-based residuals

In [ ]:
cat_detect = get_cat(1)
idxs, dists, has_xmatch = xmatch(cat_gaia["ra"], cat_gaia["dec"], cat_detect["ALPHA_J2000"], cat_detect["DELTA_J2000"], 6*u.arcsec)

In [ ]:
show_image(coadd)
x, y = coadd.wcs.world_to_pixel(coords_good_gaia.ra/u.degree, coords_good_gaia.dec / u.degree)
plt.scatter(x, y, s=0.1, color="red")

In [ ]:
show_image(coadd)
x, y = coadd.wcs.world_to_pixel(coords.ra/u.degree, coords.dec / u.degree)
plt.scatter(x, y, s=0.1, color="red")


In [ ]:
for i in range(3):

    show_image_sources(frames_orig[i], coords_good_gaia)

    plot_sources_in_wcs(frames_updated[i], coords_good_gaia, s=0.1)

In [ ]:
for i in range(3):

    fig, axs = plt.subplots(1, 2, figsize=(7, 7))
    plt.sca(axs[0])
    show_image(frames_resamp[i], log=True, fig=fig, ax=axs[0])

    x, y = frames_resamp[i].wcs.world_to_pixel(coords_good_gaia.ra/u.degree, coords_good_gaia.dec / u.degree)
    plt.scatter(x, y, s=0.1, color="red")


    plt.sca(axs[1])
    show_image_sources(frames_orig[i], coords_good_gaia, fig=fig, ax=axs[1])



In [ ]:
for i in range(3):

    show_image(frames_resamp[i], )
    x, y = frames_resamp[i].wcs.world_to_pixel(coords.ra/u.degree, coords.dec / u.degree)
    plt.scatter(x, y, s=0.1)

    # x, y = frames_updated[i].wcs.world_to_pixel(coords)
    # plt.scatter(x, y, s=0.1)


    x, y = frames_orig[i].wcs.world_to_pixel(coords)
    plt.scatter(x, y, s=0.1)

In [ ]:
for i in range(3):

    show_image_sources(frames_orig[i], )
    x, y = frames_updated[i].wcs.world_to_pixel(coords)
    plt.scatter(x, y, s=0.1, c="blue")

In [ ]:
for img in frames_orig[:3]:
    show_image(img, log=True)
    x, y = img.wcs.world_to_pixel(coords)
    plt.scatter(x, y, s=0.1, c="red")